In [452]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [453]:
# Read the CSV with latin1 encoding to avoid UnicodeDecodeError
df = pd.read_csv("../../scraper/assets/data/listings.csv", encoding="latin1")

# Fix garbled text (mojibake) in all string columns
for col in df.select_dtypes(include='object'):
    df[col] = df[col].apply(lambda x: x.encode('latin1').decode('utf-8') if isinstance(x, str) else x)



In [454]:
def try_fix_unicode(text):
    if isinstance(text, str) and '\\u' in text:
        try:
            return text.encode('utf-8').decode('unicode_escape')
        except Exception:
            return text  # if decoding fails, return original
    else:
        return text

for col in df.select_dtypes(include='object'):
    df[col] = df[col].apply(try_fix_unicode)

In [455]:
df.description[4]

'"Las imágenes del apartamento presentan un estilo arquitectónico moderno con acabados de alta calidad. Los materiales utilizados incluyen madera en los pisos y tonos neutros en las paredes, que aportan un aspecto elegante y minimalista. El mobiliario visible es escaso y de diseño sencillo, sugiriendo un enfoque contemporáneo. En cuanto a iluminación natural, hay grandes ventanales que abarcan del piso al techo, permitiendo una abundante entrada de luz, principalmente en la sala de estar que cuenta con orientación hacia una fachada urbana. La distribución del espacio es abierta, como se observa en la integración de la sala con la cocina mediante una barra que hace las veces de división. Las habitaciones parecen contar también con buena entrada de luz y vistas al entorno urbano que se muestra como cierre y con edificaciones aledañas similares. En el baño, los diseños son modernos, con acabados en porcelana y detalles en vidrio. El estado general del apartamento es de excelente conservac

In [456]:
df.isna().sum()

Link                0
Price               0
Bedrooms           25
Bathrooms          25
Area               25
Agency              0
Location            0
Datetime_Added      0
coordinates         0
administracion    745
facilities          0
upload_date         0
technical_data      0
description         0
places              0
dtype: int64

In [457]:
df['facilities'] = df['facilities'].apply(json.loads)
df['technical_data'] = df['technical_data'].apply(json.loads)
df['coordinates'] = df['coordinates'].apply(json.loads)
df['coordinates'] = df['coordinates'].apply(tuple)

df['administracion'] = df['administracion'].fillna(0)
df['Price'] = df['Price'] + df['administracion']
df['Price'] = df['Price'].astype(int)

df = df.dropna(subset=['Bedrooms'])
df = df.reset_index(drop=True)
df['Bedrooms'] = df['Bedrooms'].astype(int)
df['Bathrooms'] = df['Bathrooms'].astype(int)
df['Area'] = df['Area'].astype(int)


In [458]:
df['administracion'] = df['administracion'].fillna(0)
df['Price'] = df['Price'] + df['administracion']
df['Price'] = df['Price'].astype(int)

In [459]:

td_df = pd.json_normalize(df['technical_data'])
td_df = td_df.add_prefix('td_')
df_expanded = pd.concat([df.drop(columns=['technical_data']), td_df], axis=1)

In [460]:
df_expanded = df_expanded.dropna(subset=['td_Estrato'])
df_expanded['td_Estrato'] = df_expanded['td_Estrato'].astype(int)

In [461]:
df_expanded['td_Parqueaderos'] = df_expanded['td_Parqueaderos'].fillna(0)
df_expanded['td_Parqueaderos'] = df_expanded['td_Parqueaderos'].astype(int)


In [462]:
df_expanded['td_Piso N°'] = df_expanded['td_Piso N°'].fillna(-1)
df_expanded['td_Piso N°'] = df_expanded['td_Piso N°'].astype(int)

In [463]:
df_expanded.isna().sum()

Link                      0
Price                     0
Bedrooms                  0
Bathrooms                 0
Area                      0
Agency                    0
Location                  0
Datetime_Added            0
coordinates               0
administracion            0
facilities                0
upload_date               0
description               0
places                    0
td_Estrato                0
td_Tipo de Inmueble       0
td_Estado              1074
td_Baños                  0
td_Área Construida        0
td_Área Privada         277
td_Antigüedad            69
td_Habitaciones           0
td_Parqueaderos           0
td_Administración       725
td_Piso N°                0
td_Pisos interiores     929
dtype: int64

In [464]:
df_expanded['td_Antigüedad'].value_counts()

td_Antigüedad
1 a 8 años        329
16 a 30 años      261
9 a 15 años       234
más de 30 años    181
menor a 1 año     114
Name: count, dtype: int64

In [465]:
def parse_construction_age_range(df, col_name):
    def parse_range(value):
        if not isinstance(value, str):
            return (np.nan, np.nan)  # Por ejemplo, para NaNs o valores no string
        
        value = value.lower().strip()

        if "menor a 1 año" in value:
            return (0, 1)
        elif "más de" in value:
            try:
                num = int(value.split("más de")[1].split("año")[0].strip())
                return (num + 1, 100)  # e.g., más de 30 → (31, inf)
            except:
                return (np.nan, np.nan)
        elif "a" in value:
            try:
                parts = value.split("a")
                min_age = int(parts[0].strip())
                max_age = int(parts[1].split("año")[0].strip())
                return (min_age, max_age)
            except:
                return (np.nan, np.nan)
        else:
            return (np.nan, np.nan)  # fallback para valores inesperados

    df["construction_age_min"], df["construction_age_max"] = zip(*df[col_name].map(parse_range))
    return df

In [466]:
df_expanded = parse_construction_age_range(df_expanded, "td_Antigüedad")

df_expanded['construction_age_min'] = df_expanded['construction_age_min'].fillna(-1)
df_expanded['construction_age_min'] = df_expanded['construction_age_min'].astype(int)

df_expanded['construction_age_max'] = df_expanded['construction_age_max'].fillna(-1)
df_expanded['construction_age_max'] = df_expanded['construction_age_max'].astype(int)


In [467]:
df_expanded.isna().sum()

Link                       0
Price                      0
Bedrooms                   0
Bathrooms                  0
Area                       0
Agency                     0
Location                   0
Datetime_Added             0
coordinates                0
administracion             0
facilities                 0
upload_date                0
description                0
places                     0
td_Estrato                 0
td_Tipo de Inmueble        0
td_Estado               1074
td_Baños                   0
td_Área Construida         0
td_Área Privada          277
td_Antigüedad             69
td_Habitaciones            0
td_Parqueaderos            0
td_Administración        725
td_Piso N°                 0
td_Pisos interiores      929
construction_age_min       0
construction_age_max       0
dtype: int64

In [468]:
df_expanded.drop(columns=['td_Pisos interiores', 
                          'td_Administración', 
                          'td_Habitaciones',
                          'td_Antigüedad',
                          'td_Área Privada', 
                          'td_Área Construida',
                          'td_Baños',
                          'td_Estado',
                          'td_Tipo de Inmueble', 
                          'administracion', 
                          'Datetime_Added', 
                          'Location'], inplace=True)

In [469]:
df_expanded = df_expanded.rename(columns={'Link': 'link',
                                          'Price': 'price',
                                          'Bedrooms': 'bedrooms',
                                          'Bathrooms': 'bathrooms',
                                          'Area': 'area',
                                          'Agency': 'agency',
                                          'td_Estrato': 'stratum',
                                          'td_Parqueaderos': 'parking_lots',
                                          'td_Piso N°': 'floor',
                                          'places': 'places_input'})

In [470]:
df_expanded.isna().sum()

link                    0
price                   0
bedrooms                0
bathrooms               0
area                    0
agency                  0
coordinates             0
facilities              0
upload_date             0
description             0
places_input            0
stratum                 0
parking_lots            0
floor                   0
construction_age_min    0
construction_age_max    0
dtype: int64

In [471]:
df_expanded.shape

(1188, 16)

### Places Json

In [472]:
import json
import os
import time

from openai import OpenAI
from dotenv import load_dotenv

In [473]:

def load_env_variables(env_path=".env"):
    """
    Loads environment variables from a .env file.
    """
    load_dotenv(dotenv_path=env_path)



def extract_place_info_from_text(input_text: str, api_key: str) -> dict:
    """
    Sends a JSON string of places to OpenAI and extracts structured information.

    Parameters:
        input_text (str): A JSON string representing a list of places.
        api_key (str): Your OpenAI API key.

    Returns:
        dict: A dictionary with keys:
            - 'places': list of all 'nombre' values
            - 'location': deduplicated list of neighborhoods from 'dirección'
            - 'transportation': 'nombre' values where 'tipos' includes 
                                'bus_stop' or 'transit_station'
    """
    client = OpenAI(api_key=api_key)

    system_prompt = (
        "You are a strict JSON parser. A user will give you a JSON string representing a list of places.\n"
        "Each place has 'nombre', 'dirección', 'tipos', and 'distancia_km'.\n\n"
        "Return ONLY a JSON object with:\n"
        "- 'places': list of all 'nombre' values\n"
        "- 'location': deduplicated list of neighborhood names found in 'dirección' "
        "(e.g., 'Santa Fé', 'Chapinero', etc.)\n"
        "- 'transportation': list of 'nombre' values where 'tipos' contains 'bus_stop' or 'transit_station'\n\n"
        "You must only return a valid JSON object with those three keys. Do not explain or say anything else."
    )

    # Define structured response format with a schema
    response_format = {
        "type": "json_schema",
        "json_schema": {
            "name": "place_extraction_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "places": {
                        "type": "array",
                        "items": {"type": "string"}
                    },
                    "location": {
                        "type": "array",
                        "items": {"type": "string"}
                    },
                    "transportation": {
                        "type": "array",
                        "items": {"type": "string"}
                    }
                },
                "required": ["places", "location", "transportation"]
            }
        }
    }

    # Make request to OpenAI API
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": input_text}
        ],
        response_format=response_format
    )

    # Parse and return the JSON content
    return json.loads(response.choices[0].message.content)



In [474]:
def clean_text_for_csv(s: str) -> str:
    """
    Drop any characters that can't be UTF-8 encoded (e.g. surrogates).
    """
    if isinstance(s, str):
        return s.encode("utf-8", errors="ignore").decode("utf-8")
    return s

In [475]:
def process_and_save_places(df: pd.DataFrame, csv_path: str, api_key: str, delay_seconds: float = 1.0):
    """
    Processes each row of the CSV's 'places' column with OpenAI, extracts:
       - places_names
       - locations
       - transportation
    and saves back to the same CSV after each row, cleaning out surrogate chars.
    """

    # Ensure target columns exist
    for col in ("places", "location", "transportation"):
        if col not in df.columns:
            df[col] = None

    for i, row in df.iterrows():
        # Skip rows already processed
        if pd.notna(row["places"]):
            continue

        input_text = row["places_input"]

        try:
            result = extract_place_info_from_text(input_text, api_key)
            print(result)

            # Clean and convert to JSON strings
            places_str = clean_text_for_csv(json.dumps(result["places"], ensure_ascii=False))
            locations_str = clean_text_for_csv(json.dumps(result["location"], ensure_ascii=False))
            transport_str = clean_text_for_csv(json.dumps(result["transportation"], ensure_ascii=False))

            # Save results into DataFrame
            df.at[i, "places"] = places_str
            df.at[i, "location"] = locations_str
            df.at[i, "transportation"] = transport_str

            # Save to file after each row to avoid data loss
            df.to_csv(csv_path, index=False)

            # Respect rate limits
            time.sleep(delay_seconds)

        except Exception as e:
            print(f"[{i+1}/{len(df)}] Error on row {i}: {e}")
            continue

    print("Finished processing all rows.")

In [476]:

# Load API key from environment
load_env_variables()
api_key = os.getenv("OPENAI_API_KEY")

In [477]:
csv_path = "../assets/data/listings_clean.csv"
process_and_save_places(df= df_expanded, 
                        csv_path=csv_path, 
                        api_key=api_key)

{'places': ['San Martín Centro Comercial', 'Parque Nacional', 'Hotel Ibis Bogotá Museo', 'Parque Nacional Enrique Olaya Herrera', 'Transmilenio Calle 34', 'Parque Central Bavaria', 'La Monferrina', 'Plaza 39 Centro Comercial', 'Buffet King Bogotá San Martin', 'Éxito San Martín', 'Parque de la Independencia - Bicentenario', 'Restaurante Temático La Juguetería - Macarena', 'La Trocha, La Casa de la Paz', 'Wok Museo Nacional', 'Santa Fe Restaurante Bogotá', 'Longo´s Macarena', 'Museo Nacional', "McDonald's", 'Crepes & Waffles Centro Comercial Plaza 39', 'Crepes & Waffles Centro Internacional'], 'location': ['Bogotá', 'Chapinero', 'Santa Fé'], 'transportation': ['Transmilenio Calle 34', 'Museo Nacional']}
[1/1188] Error on row 0: 'utf-8' codec can't encode characters in position 3764-3769: surrogates not allowed
{'places': ['Terminal de Transporte S.A.', 'Hornitos La Felicidad', 'Terminal de Transportes de Bogota', 'Éxito La Felicidad', 'Industrias Vanyplas Ltda.', 'Éxito Esperanza', 'Parq

KeyboardInterrupt: 

In [ ]:
df_expanded.places[6]

'[{"nombre": "Metro Fontibón", "dirección": "Cl. 17 #113, Bogotá, Colombia", "tipos": ["supermarket", "grocery_store", "food_store", "food", "point_of_interest", "store", "establishment"], "distancia_km": 0.76}, {"nombre": "Metro Fontibón", "dirección": "Cra. 113 #17-15, Bogotá, Colombia", "tipos": ["supermarket", "grocery_store", "food_store", "food", "point_of_interest", "store", "establishment"], "distancia_km": 0.79}, {"nombre": "Parque Atahualpa", "dirección": "Cra. 113 # 23-D-01 Erróeamente IDRD indica Ac 24 No 113-51 D.C., Bogotá, Colombia", "tipos": ["park", "point_of_interest", "establishment"], "distancia_km": 0.7}, {"nombre": "Piqueteadero Guadalajara", "dirección": "Cra. 110 #18b8, Bogotá, Colombia", "tipos": ["restaurant", "food", "point_of_interest", "establishment"], "distancia_km": 0.9}, {"nombre": "Asadero La Gran Llanera Internacional", "dirección": "Calle 24, Av La Esperanza #122 95, Bogotá, Colombia", "tipos": ["restaurant", "food", "point_of_interest", "establishme

In [275]:
input_text = df_expanded.places[6]
result = extract_place_info_from_text(input_text, api_key)


In [276]:
result

{'places': ['Metro Fontibón',
  'Metro Fontibón',
  'Parque Atahualpa',
  'Piqueteadero Guadalajara',
  'Asadero La Gran Llanera Internacional',
  'Parque Público Zonal Atahualpa',
  'Merqueo Fontibón',
  'HIPERMERCADOS EDEN SAS',
  'Parque la aldea',
  'La Llanerita',
  'D1 Mercado',
  'Ara Villa Carmenza',
  'Coarali',
  'PARQUE BATAVIA',
  'Supermercado Ahorremas | Aliado Red Contigo',
  'Mercado Super Exito | Aliado Red Contigo',
  'Alimentador La Estancia Parada 12 Transmilenio',
  'Tienda D1 Atahualpa',
  'Parque Barrio La Zelfita',
  'Restaurante Yeng Sing'],
 'location': ['Fontibón', 'Bogotá'],
 'transportation': ['Alimentador La Estancia Parada 12 Transmilenio']}

In [285]:
df_expanded.places_names[0]

'["San Martín Centro Comercial", "Parque Nacional", "Hotel Ibis Bogotá Museo", "Parque Nacional Enrique Olaya Herrera", "Transmilenio Calle 34", "Parque Central Bavaria", "La Monferrina", "Plaza 39 Centro Comercial", "Buffet King Bogotá San Martin", "Éxito San Martín", "Parque de la Independencia - Bicentenario", "Restaurante Temático La Juguetería - Macarena", "La Trocha, La Casa de la Paz", "Wok Museo Nacional", "Santa Fe Restaurante Bogotá", "Longo´s Macarena", "Museo Nacional", "McDonald\'s", "Crepes & Waffles Centro Comercial Plaza 39", "Crepes & Waffles Centro Internacional"]'

## Clean Description

In [514]:
df_places = pd.read_csv(csv_path, encoding="latin1")

In [515]:
# Fix garbled text (mojibake) in all string columns
for col in df_places.select_dtypes(include='object'):
    df_places[col] = df_places[col].apply(lambda x: x.encode('latin1').decode('utf-8') if isinstance(x, str) else x)

In [516]:
for col in df_places.select_dtypes(include='object'):
    df_places[col] = df_places[col].apply(try_fix_unicode)

In [517]:
df_places['places'] = df_places['places'].apply(json.loads).tolist()
df_places['location'] = df_places['location'].apply(json.loads).tolist()
df_places['transportation'] = df_places['transportation'].apply(json.loads).tolist()

In [518]:
df_places['description'][78]

'"El apartamento presenta un estilo arquitectónico moderno con acabados sencillos; los materiales visibles como la madera en muebles de cocina y closet sugieren un diseño funcional y básico. La iluminación natural en los interiores parece limitada ya que las ventanas son de tamaño estándar situadas en la sala y las habitaciones con orientación hacia un edificio cercano lo cual puede restringir la entrada de luz y ofrecer vistas urbanas cerradas. Los espacios se distribuyen de manera abierta particularmente en la sala y la cocina que comparten un área integrada; el baño y las habitaciones se muestran como espacios cerrados e independientes. En cuanto al estado general las paredes pisos y techos están bien conservados sin signos visibles de desgaste notable y la decoración es mínima prevaleciendo superficies lisas con pequeños estantes en el baño. La limpieza general es adecuada y no se observan elementos estructurales particulares de interés especial."'

In [519]:
def summarize_property_description(input_text: str, api_key: str) -> str:
    """
    Sends a property description to OpenAI and returns a concise, natural-language summary
    using controlled vocabulary for use in embeddings.

    Parameters:
        input_text (str): A paragraph describing a property.
        api_key (str): Your OpenAI API key.

    Returns:
        str: A short, coherent summary in natural language following a controlled vocabulary.
    """
    client = OpenAI(api_key=api_key)

    system_prompt = (
        "Eres un modelo que resume descripciones inmobiliarias. "
        "Tu tarea es generar un resumen breve (menos de 700 caracteres), natural y coherente del párrafo proporcionado, "
        "usando únicamente el siguiente vocabulario controlado:\n\n"
        "- tipo_de_vista: vista panorámica, vista urbana, vista cerrada, vista interior, sin vista\n"
        "- iluminacion_natural: iluminación abundante, iluminación moderada, iluminación limitada\n"
        "- acabados: acabados lujosos, acabados modernos, acabados sencillos, acabados utilitarios, acabados básicos\n"
        "- estado_general: nuevo, bien cuidado, habitable, por renovar, en mal estado\n"
        "- distribucion: distribución abierta, distribución compartimentada, diseño tradicional, planta libre\n"
        "- entorno_exterior: entorno urbano, entorno suburbano, entorno natural, densamente construido, con áreas verdes\n"
        "- materiales_cocina: madera laminada, granito, acero inoxidable, cerámica, madera natural, melamina\n"
        "- estado_paredes_techos: en buen estado, con desgaste, con humedad, recientemente renovados\n\n"
        "No incluyas metadatos estructurados como número de habitaciones, metros cuadrados o parqueadero.\n"
        "No repitas información redundante. Solo responde con el resumen en lenguaje natural. No expliques nada."
    )

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": input_text}
        ]
    )

    return response.choices[0].message.content.strip()

In [520]:
input_text = df_expanded.description[0]
result = summarize_property_description(input_text, api_key)

In [ ]:
def process_descriptions(df: pd.DataFrame, api_key: str):

    # Crear nueva columna 'input' si no existe
    if "description_input" not in df.columns:
        df["description"] = None

    # Iterar por cada fila y generar resumen
    for i, row in df.iterrows():
        if pd.notna(row["description_input"]):  # Ya procesado
            continue

        try:
            resumen = summarize_property_description(row["description_input"], api_key)
            df.at[i, "description"] = resumen
            print(f"[{i+1}/{len(df)}] Procesado correctamente")
        except Exception as e:
            print(f"[{i+1}/{len(df)}] Error en la fila {i}: {e}")

    return df

In [522]:
df_places = df_places.rename(columns={'description': 'description_input'})

In [523]:
df_places = process_descriptions(df_places, api_key)

KeyError: 'description'